In [1]:
import chromadb
from chromadb.utils import embedding_functions
from autogen import ConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY=os.environ["GOOGLE_API_KEY"]

In [2]:
llm_config = {"model": "gemini-1.5-flash",
              "api_key": GOOGLE_API_KEY,
              "api_type": "google"}

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
genai_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(
                api_key=GOOGLE_API_KEY,
                model_name="models/embedding-001"
            )

In [3]:
retriever_config = {
    "task" : "qa",
    "model" : llm_config["model"],
    "client": chromadb.PersistentClient(path="docs/chroma"),
    "embedding_function" : genai_ef,
    "chunk_token_size": 2000,
    "vector_db": "chroma",
    "docs_path" : ["/home/shaheem/Documents/Project files/Notification system/pdf_chat/Test_Automation_questions.pdf"],
    "overwrite": False,
    "get_or_create": True 
}

In [4]:
rag_proxy_agent = RetrieveUserProxyAgent(
    name="rag_proxy_agent",
    human_input_mode="NEVER",
    retrieve_config=retriever_config,
    code_execution_config=False
)

In [5]:
rag_assistant =RetrieveAssistantAgent(
    name="rag_assistant",
    system_message="You are a helpful AI assistant that retrives the context",
    llm_config={
        "timeout":600,
        "config_list":[llm_config]
    }
) 

In [6]:
query = "What are the common causes of Device Failure"

In [7]:
chat_result=rag_proxy_agent.initiate_chat(rag_assistant,
                              problem=query,
                              n_results=1,
                              message=rag_proxy_agent.message_generator,
                              max_turns=3)

2024-09-04 19:33:36,267 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.


Trying to create collection.


2024-09-04 19:33:36,567 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 1 chunks.
Model gemini-1.5-flash not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['938b583d']]
Adding content of doc 938b583d to context.
rag_proxy_agent (to rag_assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What are the common causes of Device Failure

Context is: Device Failure scenarios for the systems
1. Polar
Failure case:- Direct power source is power off or removed
Question :- Is the device's adapter AC current connected?
Failure case:- Temparature sensor is malfunctioning ○ Question :- Is there a potal solwing sensor error? ○ Question :- Is sensor wire damaged?
2. Common
Failure case:- SIM damaged
Question :- Is water present inside the sensor/are there rust inside the sensor circuitry
Failure case:- GSM module damaged
Question :- Is there an LED blinking w

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


...

VectorDB returns doc_ids:  [['938b583d']]


Number of requested results 7 is greater than number of elements in index 1, updating n_results = 1


VectorDB returns doc_ids:  [['938b583d']]


Number of requested results 9 is greater than number of elements in index 1, updating n_results = 1


VectorDB returns doc_ids:  [['938b583d']]
No more context, will terminate.
rag_proxy_agent (to rag_assistant):

TERMINATE

--------------------------------------------------------------------------------


In [46]:
#rag_proxy_agent.last_message()["content"]
print(rag_assistant.last_message()['content'])

Direct power source is power off or removed, temperature sensor is malfunctioning, SIM damaged, GSM module damaged, GSM module loss connection with main board. 



In [31]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': "You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the\ncontext provided by the user.\nIf you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser's question is: What are the possible causes of Device Failure\n\nContext is: Device Failure scenarios for the systems\n1. Polar\nFailure case:- Direct power source is power off or removed\nQuestion :- Is the device's adapter AC current connected?\nFailure case:- Temparature sensor is malfunctioning ○ Question :- Is there a potal solwing sensor error? ○ Question :- Is sensor wire damaged?\n2. Common\nFailure case:- SIM damaged\nQuestion :- Is water present inside the sensor/are there rust inside the sensor circuitry\nFailure case:- GSM module damaged\nQuestion :- Is there an LED blinking within the device? ○ Question :- Is there water insid